# Diagrama:

![1](https://user-images.githubusercontent.com/47722525/82421650-e9da2f80-9a46-11ea-942d-99a9f067de22.png)

# Código:

In [3]:
class MomentoVigaSimple(object):
    def __init__(self, base, peralte, AreaAcero, ResistConcreto, ResistAcero):
        self.b = base # cm
        self.d = peralte # cm
        self.As = AreaAcero # cm2
        self.fc = ResistConcreto # kg/cm2
        self.fy = ResistAcero # kg/cm2
        self.eu = 0.003 # Adimencional
        self.Es = 2*10**6 # kg/cm2
        self.Bu = 0 # Adimencional
        
    def Beta_1(self):
        aux = 1.05-(self.fc/1400)
        if (0.65<= aux) and (aux<=0.85):
            return aux
        elif (aux < 0.65):
            return 0.65
        elif (aux > 0.85):
            return 0.85 
    
    def Const_c(self):
        Bu = self.Beta_u()
        return Bu*self.d
    
    def Const_a(self,c):
        B1 = self.Beta_1()
        return B1*c
    
    def Compresion(self,Bu):
        B1 = self.Beta_1()
        return 0.85*self.fc*B1*Bu*self.d*self.b # kg
    
    def Tension(self,Bu):
        es = (1-Bu)*self.eu/Bu
        ey = self.fy/self.Es
        if es > ey:
            fs = self.fy
        else:
            fs = es*self.Es
        return self.As*fs
    
    def Beta_u(self):
        xi = 0.0001
        xf = 1
        cont = 0
        fx = lambda Bu: self.Compresion(Bu)-self.Tension(Bu)
        # Método de la bisección
        while (cont <= 1000):
            xr = (xi+xf)/2
            ea = abs(xf-xi)*100/xi
            if fx(xi)*fx(xr)<0:
                xf = xr
            elif fx(xi)*fx(xr)>0:
                xi = xr
            elif (fx(xi)*fx(xr)==0 or ea<=0.0001):
                break
            cont+=cont
        self.Bu = xr
        return xr
    
    def Palanca(self):
        c = self.Const_c() 
        a = self.Const_a(c) 
        return self.d - a/2
    
    def Momento(self,fi = 1):
        z = self.Palanca() 
        T = self.Tension(self.Bu) 
        return round(T*z*fi/100000,2) 
    

# Ejemplo:

## Determinar la resistencia a flexión de la siguiente sección rectangular:

![3](https://user-images.githubusercontent.com/47722525/82421693-fd859600-9a46-11ea-9af9-80f40eb64e0d.png)

### Datos:

In [42]:
base = 25 #cm
peralte = 55 #cm
AreaAcero = 20.28 #cm2
ResistConcreto = 250 #kg/cm2
ResistAcero = 4200 #kg/cm2

### Solución:

In [43]:
MVS = MomentoVigaSimple(base, peralte, AreaAcero, ResistConcreto, ResistAcero)

### Resistencia a flexión:

In [49]:
print('La resistencia a flexion es: ',MVS.Momento(), 'Ton-m')

La resistencia a flexion es:  40.02 Ton-m


### Momento Nominal:

In [50]:
fi = 0.9 
print('El Momento Nominal es: ',MVS.Momento(fi), 'Ton-m')

El Momento Nominal es:  36.02 Ton-m


### Valor de Beta_u:

In [46]:
Bu = MVS.Beta_u()
print('El valor de Beta_u es: ',Bu)

El valor de Beta_u es:  0.342954639823844


### Fuerza de Compresión:

In [51]:
C = MVS.Compresion(Bu)
print('La Fuerza de Compresión es: ',C, 'kg')

La Fuerza de Compresión es:  85176.0 kg


### Fuerza de Tensión:

In [52]:
T = MVS.Tension(Bu)
print('La Fuerza de Tensión es: ',T, 'kg')

La Fuerza de Tensión es:  85176.0 kg


### Vemos que se cumple la relación:

![2](https://user-images.githubusercontent.com/47722525/82421740-155d1a00-9a47-11ea-939b-5f984b22e572.png)